# PowerPoint to Voiceover Transcript Generator

This notebook demonstrates the complete workflow for converting PowerPoint presentations into AI-generated voiceover transcripts using Llama 4 Maverick through the Llama API.

## Overview

This workflow performs the following operations:

1. **Content Extraction**: Pulls speaker notes and visual elements from PowerPoint slides
2. **Image Conversion**: Transforms slides into high-quality images for AI analysis
3. **Transcript Generation**: Uses Llama vision models to create natural-sounding voiceover content
4. **Speech Optimization**: Converts numbers, technical terms, and abbreviations to spoken form
5. **Results Export**: Saves transcripts in multiple formats for further use

## Prerequisites

Before running this notebook, ensure you have:
- Created a `.env` file with your `LLAMA_API_KEY`
- Updated `config.yaml` with your presentation file path
---

## Setup and Configuration

Import required libraries and load environment configuration.

In [ ]:
# Import required libraries
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from IPython.display import display

# Load environment variables from .env file
load_dotenv()

# Verify setup
if os.getenv('LLAMA_API_KEY'):
    print("SUCCESS: Environment loaded successfully!")
    print("SUCCESS: Llama API key found")
else:
    print("WARNING: LLAMA_API_KEY not found in .env file")
    print("Please check your .env file and add your API key")

In [ ]:
# Import custom modules
try:
    from src.core.pptx_processor import extract_pptx_notes, pptx_to_images_and_notes
    from src.processors.transcript_generator import process_slides, TranscriptProcessor
    from src.config.settings import load_config, get_config

    print("SUCCESS: All modules imported successfully!")
    print("- PPTX processor ready")
    print("- Transcript generator ready")
    print("- Configuration manager ready")

except ImportError as e:
    print(f"ERROR: Import error: {e}")
    print("Make sure you're running from the project root directory")

In [ ]:
# Load and display configuration
config = load_config()
print("SUCCESS: Configuration loaded successfully!")
print("\nCurrent Settings:")
print(f"- Llama Model: {config['api']['llama_model']}")
print(f"- Image DPI: {config['processing']['default_dpi']}")
print(f"- Image Format: {config['processing']['default_format']}")

In [ ]:
# Configure file paths from config.yaml
pptx_file = config['current_project']['pptx_file'] + config['current_project']['extension']
output_dir = config['current_project']['output_dir']

print("File Configuration:")
print(f"- Input File: {pptx_file}")
print(f"- Output Directory: {output_dir}")

# Verify input file exists
if Path(pptx_file).exists():
    file_size = Path(pptx_file).stat().st_size / 1024 / 1024
    print(f"- SUCCESS: Input file found ({file_size:.1f} MB)")
else:
    print(f"- ERROR: Input file not found: {pptx_file}")
    print("  Please update the 'pptx_file' path in config.yaml")

# Create output directory if needed
Path(output_dir).mkdir(parents=True, exist_ok=True)
print(f"- SUCCESS: Output directory ready")

---
## Processing Pipeline

Execute the main processing pipeline in three key steps.

### Step 1: Extract Content and Convert to Images

Extract speaker notes and slide text, then convert the presentation to high-quality images for AI analysis.

In [ ]:
print("PROCESSING: Converting PPTX to images and extracting notes...")

result = pptx_to_images_and_notes(
    pptx_path=pptx_file,
    output_dir=output_dir,
    extract_notes=True
)

notes_df = result['notes_df']
image_files = result['image_files']

print(f"\nSUCCESS: Processing completed successfully!")
print(f"- Processed {len(image_files)} slides")
print(f"- Images saved to: {result['output_dir']}")
print(f"- Found notes on {notes_df['has_notes'].sum()} slides")
print(f"- DataFrame shape: {notes_df.shape}")

# Show sample data
print("\nSample Data (First 5 slides):")
display(notes_df[['slide_number', 'slide_title', 'has_notes', 'notes_word_count', 'slide_text_word_count']].head())

### Step 2: Generate AI Transcripts

Use the Llama vision model to analyze each slide image and generate natural-sounding voiceover transcripts.

This process:
- Analyzes slide visual content using AI vision
- Combines slide content with speaker notes
- Generates speech-optimized transcripts
- Converts numbers and technical terms to spoken form

In [ ]:
print("PROCESSING: Starting AI transcript generation...")
print(f"- Processing {len(notes_df)} slides")
print(f"- Using model: {config['api']['llama_model']}")
print("- This may take several minutes...")

# Initialize processor and generate transcripts
processor = TranscriptProcessor()
processed_df = processor.process_slides_dataframe(
    df=notes_df,
    output_dir=output_dir
)

print(f"\nSUCCESS: Transcript generation completed!")
print(f"- Generated {len(processed_df)} transcripts")
print(f"- Average length: {processed_df['ai_transcript'].str.len().mean():.0f} characters")
print(f"- Total words: {processed_df['ai_transcript'].str.split().str.len().sum():,}")

### Step 3: Save Results

Save results in multiple formats for different use cases.

In [ ]:
print("PROCESSING: Saving results in multiple formats...")

# Create output directory
os.makedirs(output_dir, exist_ok=True)

# Save complete results with all metadata
output_file = f"{output_dir}processed_transcripts.csv"
processed_df.to_csv(output_file, index=False)
print(f"- SUCCESS: Complete results saved to {output_file}")

# Save transcript-only version for voiceover work
transcript_only = processed_df[['slide_number', 'slide_title', 'ai_transcript']]
transcript_file = f"{output_dir}transcripts_only.csv"
transcript_only.to_csv(transcript_file, index=False)
print(f"- SUCCESS: Transcripts only saved to {transcript_file}")

# Save as JSON for API integration
json_file = f"{output_dir}transcripts.json"
processed_df.to_json(json_file, orient='records', indent=2)
print(f"- SUCCESS: JSON format saved to {json_file}")

# Summary statistics
total_words = processed_df['ai_transcript'].str.split().str.len().sum()
reading_time = total_words / 150  # Assuming 150 words per minute

print(f"\nExport Summary:")
print(f"- Total slides processed: {len(processed_df)}")
print(f"- Slides with speaker notes: {processed_df['has_notes'].sum()}")
print(f"- Total transcript words: {total_words:,}")
print(f"- Average transcript length: {processed_df['ai_transcript'].str.len().mean():.0f} characters")
print(f"- Estimated reading time: {reading_time:.1f} minutes")

---
# Completion Summary

## Successfully Generated:
- **Slide Images**: High-resolution images for AI analysis
- **AI Transcripts**: Speech-optimized voiceover content
- **Multiple Formats**: CSV, JSON exports for different use cases
- **Analysis**: Visual insights into content distribution and quality

## Output Files:
- `processed_transcripts.csv` - Complete dataset with all metadata
- `transcripts_only.csv` - Just slide numbers, titles, and transcripts
- `transcripts.json` - JSON format for API integration
- Individual slide images in PNG/JPEG format

## Next Steps:
1. **Review** generated transcripts for accuracy and tone
2. **Edit** any content that needs refinement
3. **Create** voiceover recordings or use TTS systems
4. **Integrate** JSON data into your video production workflow

## Tips for Better Results:
- **Rich Speaker Notes**: Slides with detailed notes generate better transcripts
- **Clear Visuals**: High-contrast slides with readable text work best
- **Consistent Style**: Maintain consistent formatting across your presentation
- **Review & Edit**: Always review AI-generated content before final use

---